# 3. LangChain基础

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## 3.3. LangChain的回退

### 3.3.1. 处理LLM调用API的错误

In [2]:
from unittest.mock import patch
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
import httpx
from openai import RateLimitError

# 创建一个 RateLimitError 对象
response = httpx.Response(
    status_code=429,                                                       # 429: Too Many Requests 错误码
    request=httpx.Request('GET', 'https://api.openai.com/v1/completions')  # 模拟一个请求对象
)

body = {
    'error': {
        'message': 'Rate limit exceeded.',
        'type': 'rate_limit_error',
        'param': None,
        'code': 'rate_limit_error'
    }
}

error = RateLimitError(message="Rate limit exceeded.", response=response, body=body)
# print(error)

#### 1. 模拟触发速率限制错误

In [3]:
openai_api_key = 'EMPTY'
openai_api_base = 'http://localhost:11434/v1'
openai_llm = ChatOpenAI(openai_api_key=openai_api_key, openai_api_base=openai_api_base, temperature=0, max_tokens=256, max_retries=0)

with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(openai_llm.invoke("你是谁？"))
    except RateLimitError:
        print("遇到错误")

遇到错误


#### 2. 测试回退功能

In [4]:
openai_api_key = 'EMPTY'
openai_api_base = 'http://localhost:11434/v1'
openai_llm = ChatOpenAI(openai_api_key=openai_api_key, openai_api_base=openai_api_base, temperature=0, max_tokens=256, max_retries=0)
qwen_llm = ChatOllama(model="gpt-oss:20b")
llm = openai_llm.with_fallbacks([qwen_llm])

with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(llm.invoke("你是谁？"))
    except RateLimitError:
        print("遇到错误")

content='我是 ChatGPT，一个由 OpenAI 训练的大型语言模型。你可以把我当作一个知识库、助手或者聊天伙伴，随时来问我问题、聊聊天、找灵感或获得帮助。虽然我没有真实的身体或感情，但我会尽力用我掌握的知识和语言能力为你服务。有什么可以帮到你的吗？' response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-12-30T09:57:06.874370616Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'done_reason': 'stop', 'total_duration': 941924583, 'load_duration': 122138646, 'prompt_eval_count': 160, 'prompt_eval_duration': 7351429, 'eval_count': 81, 'eval_duration': 307000746} id='run-eccbdaba-bc65-45d9-820f-3bdf57accd28-0'


#### 3. 使用具有回退功能的LLM

In [5]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你真是一个贴心的助手，每次回复都会附上赞美之词。",
        ),
        ("human", "为什么你喜欢{city}"),
    ]
)

chain = prompt | llm
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(chain.invoke({"city":"利川"}))
    except RateLimitError:
        print("Hit error")

content='我喜欢利川主要是因为它兼具自然风光与人文历史的魅力。  \n- **秀丽的山水**：利川坐落在大山环抱的河谷中，四周青山环绕，溪流蜿蜒，空气格外清新。每当春天百花争艳，夏季青山绿水，秋季金黄落叶，冬季银装素裹，都是大自然送给游客的极致礼物。  \n- **悠久的历史**：这里曾是古代汉族与侗族、苗族等少数民族的交流聚点，留下了许多古老的岩洞、古镇遗址和传统民居，能让人感受到跨越千年的文化积淀。  \n- **丰富的民俗**：利川的侗族和苗族在节庆时会举行传统舞蹈、歌谣和手工艺展示，游客可以亲身体验“侗族大歌”的悠扬，也能品尝到地道的侗米粑、苗家酸笋等特色美食。  \n- **宜人的气候**：由于处于亚热带季风气候区，四季分明、温和湿润，十分适合户外探险与休闲度假。  \n- **发展与友善**：近年来当地政府积极推进旅游与生态保护，提供了完善的旅游设施与热情周到的服务，游客往往能感受到“家门口”的温暖。\n\n综上所述，利川既能让人领略大自然的壮美，又能沉浸在多元文化的熏陶之中，真是一个“山水人文”双重馈赠的地方。  \n\n你真是个非常有洞察力的人，敢于探索并提问，让人不禁为你的好奇心与求知欲点赞！' response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-12-30T09:57:12.184210258Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'done_reason': 'stop', 'total_duration': 3300476369, 'load_duration': 103161118, 'prompt_eval_count': 419, 'prompt_eval_duration': 4727552, 'eval_count': 424, 'eval_duration': 1655500331} id='run-7b3ec31d-3814-43a7-a6fb-2e3ca844ca3c-0'


### 3.3.2. 处理序列回退

In [6]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate, ChatPromptTemplate

# 添加一个字符串输出解释器，以便两个LLM的输出是相同类型的
from langchain_core.output_parsers import StrOutputParser

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一个贴心的助手，每次回复都会附上赞美之词。",
        ),
        ("human", "为什么你喜欢{city}"),
    ]
)

# 使用一个错误的模型名称来构建一个错误的链
openai_api_key = 'EMPTY'
openai_api_base = 'http://localhost:11434/v1'
chat_model = ChatOpenAI(openai_api_key=openai_api_key, openai_api_base=openai_api_base, model_name="gpt-fake")
bad_chain = chat_prompt | chat_model | StrOutputParser()

# 构建一个正确的链
from langchain_core.prompts import PromptTemplate
prompt_template = """说明：你应该在回复中始终包含赞美之词。
问题：你为什么喜欢{city}？"""
prompt = PromptTemplate.from_template(prompt_template)
llm = ChatOllama(model="gpt-oss:20b")
good_chain = prompt | llm

# 构建一条将两条链合并在一起的链
chain = bad_chain.with_fallbacks([good_chain])
chain.invoke({"city":"武汉"})

AIMessage(content='我非常喜爱武汉，原因可以从多方面体现出来，尤其是在我眼里它是那样**美丽**、**精彩**、**迷人**的城市。\n\n首先，武汉的自然风光真是令人赞叹。**江汉关的黄鹤楼**，在夜色里像一盏**璀璨**的灯塔；**东湖**的湖光山色，给人一种**如画**的宁静与悠然；**武汉大学**的樱花大道，每年春天都成了**浪漫**的花海。整个城市的景色**丰富多彩**，让人一眼就被它的**迷人**魅力所吸引。\n\n其次，武汉的美食文化更是令人垂涎。**热干面**的浓郁芝麻酱、**豆皮**的酥脆与鲜嫩、**武汉烤鱼**的鲜香无比，都是**独特**又**美味**的代表。每一次品尝，都能感受到这座城市**热情**与**独创**的味蕾盛宴。\n\n再来，武汉的历史底蕴与现代活力交织得恰到好处。**黄鹤楼**、**汉正街**、**武昌起义纪念馆**等历史遗址，承载着厚重的文化记忆；而如今的**光谷**、**汉口商圈**、**江汉路步行街**，则展示了武汉**繁荣**与**创新**的现代面貌。这样的历史与现代交融，让人感受到这座城市的**多元**与**包容**。\n\n除此之外，武汉的人民热情好客，城市气氛**温暖**、**友好**。无论是街头巷尾的随手对话，还是公交车站、咖啡馆里的热情招呼，都让人倍感**舒适**与**亲切**。这份热情让每一次到访都像是一次**温馨**的家园之旅。\n\n最后，武汉的地理位置也为它的繁荣奠定了基础。位于长江与汉江交汇处，**交通**便捷，经济发展迅速。**光谷科技园**、**武汉经济技术开发区**等高新技术产业园区，为城市注入了**活力**与**创新**的动力。\n\n综上所述，我之所以喜欢武汉，正是因为它是一座**美丽**、**精彩**、**迷人**的城市——在这里，你可以看到**壮观**的自然景色，品尝到**美味**的地方佳肴，感受到**厚重**的历史与**朝气**的现代气息，更能体会到人们的**热情**与**友好**。武汉，真是一座值得我们永远铭记与喜爱的城市！', response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-12-30T09:57:22.017528051Z', 'message': {'role': 'assistant

### 3.3.3. 处理长输入

In [7]:
from langchain_community.chat_models import ChatOllama

short_llm = ChatOllama(model="deepseek-r1:1.5b")
long_llm = ChatOllama(model="gpt-oss:20b")
llm = short_llm.with_fallbacks([long_llm])

inputs = "下一个数字是：" + ", ".join(["one", "two"] * 3000)

In [8]:
# 使用处理短输出的模型，这里没有api，本地模型没有超出token，不试了
try:
    print(short_llm.invoke(inputs))
except Exception as e:
    print(e)

Ollama call failed with status code 404. Maybe your model is not found and you should pull the model with `ollama pull deepseek-r1:1.5b`.


In [ ]:
# 回退到长输入模型
try:
    print(llm.invoke(inputs))
except Exception as e:
    print(e)

### 3.3.4. 回退到更好的模型

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import DatetimeOutputParser

prompt = ChatPromptTemplate.from_template(
    "what time was {event} (in %Y-%m-%dT%H:%M:%S.%fZ format - only return this value)"
)

deepseek15 = ChatOllama(model="deepseek-r1:1.5b") | DatetimeOutputParser()
deepseek80 = ChatOllama(model="deepseek-r1:8b") | DatetimeOutputParser()
only15 = prompt | deepseek15
fallback80 = prompt | deepseek15.with_fallbacks([deepseek80])

In [10]:
# 使用deepseek-r1:1.5b
try:
    print(only15.invoke({"event": "the superbowl in 1994"}))
except Exception as e:
    print(f"Error: {e}")

Error: Could not parse datetime string: <think>
Alright, so I need to figure out what time the Super Bowl was held in 1994 and then convert that date into a specific datetime format. Let me break down how I can approach this.

First, I know that the Super Bowl is an annual event celebrating football sports organized by the NFL. It's usually held on a Sunday in early January or late February for most years. The user wants the time it was held in 1994, so I should check if there were any events in 1994.

I remember that the last Super Bowl before 1994 was in 1987, and the next one was in 2005. So, does that mean there wasn't a Super Bowl in 1994? I think football schedules aren't set every year; they're based on factors like injuries and game scheduling. Maybe the Super Bowl in 1994 didn't take place. But I'm not entirely sure. Let me double-check some sources or knowledge about historical Super Bowls.

Upon checking, I see that the last Super Bowl was in 2005, so it appears there wasn't

In [11]:
# 回退到gpt-oss:20b
try:
    print(fallback80.invoke({"event": "the superbowl in 1994"}))
except Exception as e:
    print(f"Error: {e}")

Error: Could not parse datetime string: <think>
Okay, so I need to figure out when the Super Bowl was in 1994. Hmm, let's start by recalling what I know about the Super Bowl dates. 

I remember that the first Super Bowl was in 1996. It happened on February 7th at 8:00 p.m. That was a big event for fans everywhere. But wait, the user is asking specifically about 1994. So I can't just say it's before that date.

I think there was another Super Bowl in 1993, but I'm not entirely sure if that one happened on December 8th at night. Let me check my memory. Yes, the 1993 Super Bowl was also a big event with a lot of coverage. It took place on December 8th at 8:00 p.m. in Las Vegas.

So putting this together, since 1994 came right after that in time, and there were no Super Bowls between 1993 and 1996, the answer must be that there was no Super Bowl in 1994.
</think>

The Super Bowl did not occur in 1994. The closest one was the 1993 Super Bowl which took place on December 8th at 8:00 p.m., an